# summary statistics

We now need to merge our files


-----------------
##  SUMMARY STATS  
> WARNING: (🚨🚨🚨🚨 WIP)
### normalizations.

- overlaps, normalized by CYTOPLASM, A, and B
- per cell averages, medians, std, and totals

These is all pandas munging and very straightforward tabular manipulation.


In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union, Dict, List
import itertools 
import glob

import time

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files,
                                        read_tiff_image)



from infer_subc.constants import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *
from infer_subc.utils.stats import _assert_uint16_labels
from infer_subc.core.img import label_uint16


import time
%load_ext autoreload
%autoreload 2



#### random code i found that may be helpful

In [ ]:
# mito_table.volume.mean()

# titanic.groupby(["Sex", "Pclass"])["Fare"].mean() #I think Fare is what is being averaged 

# something.groupby(['celltype', 'condition'])['Volume'].mean() # like maybe something like this
# something.groupby(['celltype', 'condition'])['Volume', ].mean() # i wonder if this will work

# ##### I think this one gives you the number of things in the category you choose:
# something["Pclass"].value_counts()

Other resource pages:

https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html#pandas.DataFrame.agg

https://pandas.pydata.org/docs/user_guide/groupby.html#groupby

In [53]:
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts\Infer-subc"

quant_data_path = data_root_path / "quant"

str(quant_data_path)

'C:\\Users\\Shannon\\Documents\\Python_Scripts\\Infer-subc\\quant'

__________________
## 1. Read in the csv files

In [54]:
test_collect_csvs = sorted(quant_data_path.glob("*.csv"))
test_collect_csvs

[WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/quant/20230922_TEST_withregions_contacts.csv'),
 WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/quant/20230922_TEST_withregions_distributions.csv'),
 WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/quant/20230922_TEST_withregions_organelles.csv'),
 WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/quant/20230922_TEST_withregions_regions.csv')]

In [61]:
org_tabs = []
contact_tabs = []
dist_tabs = []
region_tabs = []
for stats_f in test_collect_csvs:
    stem = stats_f.stem.split("-")[0]
    print(stem)

    org = "organelles"
    contacts = "contacts"
    dist = "distributions"
    regions = "_regions"

    if org in stem:
        test_orgs = pd.read_csv(stats_f, index_col=0)
        test_orgs.insert(0, "dataset", stem)
        org_tabs.append(test_orgs)
    if contacts in stem:
        test_contact = pd.read_csv(stats_f, index_col=0)
        test_contact.insert(0, "dataset", stem)
        contact_tabs.append(test_contact)
    if dist in stem:
        test_dist = pd.read_csv(stats_f, index_col=0)
        test_dist.insert(0, "dataset", stem)
        dist_tabs.append(test_dist)
    if regions in stem:
        test_regions = pd.read_csv(stats_f, index_col=0)
        test_regions.insert(0, "dataset", stem)
        region_tabs.append(test_regions)

region_tabs[0]

20230922_TEST_withregions_contacts
20230922_TEST_withregions_distributions
20230922_TEST_withregions_organelles
20230922_TEST_withregions_regions


,dataset,image_name,region,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity-0,min_intensity-1,min_intensity-2,min_intensity-3,min_intensity-4,min_intensity-5,max_intensity-0,max_intensity-1,max_intensity-2,max_intensity-3,max_intensity-4,max_intensity-5,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3,mean_intensity-4,mean_intensity-5,standard_deviation_intensity-0,standard_deviation_intensity-1,standard_deviation_intensity-2,standard_deviation_intensity-3,standard_deviation_intensity-4,standard_deviation_intensity-5
0,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,cell,1,6.197872,428.801250,335.530607,2,0,0,12,704,704,1385256.0,433195.500000,138.306887,0.279502,5,2228899.0,0.621498,683.754058,0.0,0.0,0.0,0.0,0.0,0.0,27528.0,33141.0,65535.0,62085.0,36701.0,46708.0,403.168213,692.458477,771.805308,779.481645,1947.697575,1274.262327,1407.065577,2000.562999,3166.995343,2036.527360,2567.820770,2881.159265
1,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,cyto,1,6.219782,433.585791,345.428902,2,0,0,12,704,704,1200676.0,509387.375000,131.868906,0.242259,6,2228899.0,0.538686,716.709526,0.0,0.0,0.0,0.0,0.0,0.0,27528.0,33141.0,65535.0,62085.0,36701.0,46708.0,425.212126,787.113542,871.315025,871.040081,2194.334357,1437.301181,1496.487741,2131.404180,3388.818724,2168.966451,2670.335772,3061.142759
2,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,nuc,2,6.040147,395.785736,270.318574,2,279,161,12,507,386,261616.0,82129.609375,79.351516,0.509973,1,308364.0,0.848400,249.050492,0.0,0.0,0.0,0.0,0.0,0.0,12723.0,5034.0,5905.0,10281.0,8669.0,5118.0,265.768382,79.491155,128.359301,244.177481,505.640878,248.953401,539.561569,227.205225,303.515298,472.228979,642.459358,279.599798
3,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_3_Unmixing,cell,1,8.008865,327.122783,362.547316,2,37,26,15,662,704,1524566.0,446066.468750,142.795943,0.276754,-11,2694300.0,0.565849,543.021915,0.0,0.0,0.0,0.0,0.0,0.0,12116.0,16304.0,65535.0,55322.0,39102.0,44187.0,208.885957,372.714601,702.209159,782.494696,1795.936843,890.277209,563.562522,1007.714497,2917.406918,1654.605360,2215.850078,2017.096597
4,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_3_Unmixing,cyto,1,8.151382,340.566871,360.544005,2,37,26,15,662,704,1127826.0,541597.875000,129.146060,0.204734,-10,2694300.0,0.418597,600.987883,0.0,0.0,0.0,0.0,0.0,0.0,12116.0,16304.0,65535.0,55322.0,39102.0,44187.0,159.991511,477.686593,901.542501,923.539096,2115.190216,1108.037031,498.502926,1143.453274,3362.518542,1853.968275,2389.977348,2294.730076
5,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_3_Unmixing,nuc,4,7.607422,288.263446,368.279921,2,166,241,15,399,508,490764.0,100309.101562,97.864390,0.606823,1,528842.0,0.927997,289.646490,0.0,0.0,0.0,0.0,0.0,0.0,7186.0,5685.0,17793.0,34640.0,18218.0,13415.0,338.601768,79.439496,149.860377,449.376215,1124.335094,334.590522,677.572541,263.462604,414.816605,838.926292,1452.374604,471.895043
6,20230922_TEST_withregions_regions,a24hrs-Ctrl +oleicAcid 50uM_10_Unmixing,cell,1,10.399536,418.503591,277.745075,3,16,101,20,680,620,1308173.0,305086.531250,135.692399,0.223296,-3,2037599.0,0.642017,530.693387,0.0,0.0,0.0,0.0,0.0,0.0,13243.0,24004.0,46100.0,50873.0,28711.0,31691.0,290.738296,439.340372,393.455995,661.129718,1437.561406,617.630372,750.983615,1362.345476,1394.062782,1652.069807,1885.350797,1195.399005
7,20230922_TEST_withregions_regions,a24hrs-Ctrl +oleicAcid 50uM_10_Unmixing,cyto,1,10.328079,413.678126,292.812644,3,16,101,20,680,620,955362.0,375950.781250,122.195847,0.163074,-2,2037599.0,0.468867,592.228232,0.0,0.0,0.0,0.0,0.0,0.0,13243.0,24004.0,46100.0,50873.0,28711.0,31691.0,198.936236,573.565322,492.175555,818.724918,1786.856129,768.632770,626.510210,1563.359948,1603.555001,1882.571817,2043.05982

In [81]:
org_df = pd.concat(org_tabs,axis=0, join='outer')
contacts_df = pd.concat(contact_tabs,axis=0, join='outer')
dist_df = pd.concat(dist_tabs,axis=0, join='outer')
regions_df = pd.concat(region_tabs,axis=0, join='outer')

### Add in a step to collect from multiple file locations

In [ ]:
test_multi_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc/quant"

csv_path_list = [test_multi_root_path/"test_loc_1", test_multi_root_path/"test_loc_2" ]

csv_path_list

In [ ]:
for loc in csv_path_list:
    files_store = sorted(loc.glob("*.csv"))
    for file in files_store:
        stem = file.stem.split("-")[0]
        print(stem)

_______________________
## 2. Adding new columns to the tables

In [18]:
pd.set_option('display.max_columns', None)

#### Summarize the contact type based on org_a and org_B labels so that it can fit into the "organelle" column in the per cell data table

In [82]:
contacts_df.insert(3, "organelle", (contacts_df['org_A'] + '_' + contacts_df['org_B']))

#### Surface area to volume ratio

In [83]:
org_df.insert(15, "sa_volume_ratio", org_df["surface_area"].div(org_df["volume"]))
contacts_df.insert(20, "sa_volume_ratio", contacts_df["surface_area"].div(contacts_df["volume"]))
regions_df.insert(15, "sa_volume_ratio", contacts_df["surface_area"].div(contacts_df["volume"]))

In [84]:
org_df

,dataset,image_name,organelle,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,sa_volume_ratio,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity,mito_overlap,mito_labels,golgi_overlap,golgi_labels,perox_overlap,perox_labels,ER_overlap,ER_labels,LD_overlap,LD_labels,lyso_overlap,lyso_labels
0,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,3,2.445946,408.871622,361.472973,2,404,356,4,415,367,148.0,259.625702,1.754228,6.562778,0.611570,1,164.0,0.902439,13.198776,0.0,10274.0,3352.513514,2409.334613,0.0,[],0.0,[],0.0,[],114.0,[1],0.0,[],NaN,NaN
1,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,4,2.506410,442.634615,233.070513,2,435,228,4,452,239,156.0,393.570221,2.522886,6.678957,0.417112,1,238.0,0.655462,20.102100,0.0,12723.0,4483.294872,3374.768226,0.0,[],0.0,[],0.0,[],101.0,[1],0.0,[],NaN,NaN
2,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,6,2.307692,338.646154,345.938462,2,335,342,4,343,351,65.0,155.116577,2.386409,4.988518,0.451389,1,81.0,0.802469,9.003436,0.0,6547.0,1794.353846,1586.629194,0.0,[],0.0,[],0.0,[],65.0,[1],0.0,[],NaN,NaN
3,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,9,2.000000,424.729167,567.520833,2,421,564,3,429,572,48.0,143.774887,2.995310,4.509007,0.750000,1,0.0,inf,9.459070,52.0,6220.0,2228.770833,1581.970583,0.0,[],0.0,[],0.0,[],10.0,[1],0.0,[],NaN,NaN
4,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,10,3.321348,434.123596,412.208989,2,427,406,6,443,419,445.0,498.599670,1.120449,9.472264,0.534856,1,545.0,0.816514,15.586267,0.0,14538.0,5215.573034,2812.248788,40.0,[2],0.0,[],0.0,[],32.0,[1],0.0,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,perox,97,13.000000,364.000000,254.285714,13,363,253,14,366,256,7.0,31.170115,4.452874,2.373376,0.777778,1,0.0,inf,3.380617,12025.0,23903.0,19413.000000,3858.349240,0.0,[],0.0,[],NaN,NaN,2.0,[1],0.0,[],0.0,[]
3495,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,perox,98,15.000000,166.250000,349.000000,15,166,348,16,168,351,4.0,21.513260,5.378315,1.969490,0.666667,1,0.0,inf,3.162278,17600.0,27696.0,20437.750000,4198.654926,0.0,[],0.0,[],NaN,NaN,4.0,[1],0.0,[],0.0,[]
3496,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,perox,99,15.000000,329.800000,331.500000,15,329,330,16,332,334,10.0,40.826969,4.082697,2.673009,0.833333,1,0.0,inf,4.582576,6664.0,27020.0,18314.600000,6318.841796,0.0,[],0.0,[],NaN,NaN,0.0,[],0.0,[],0.0,[]
3497,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,ER,1,7.789589,343.951855,278.909246,2,80,106,18,682,515,254645.0,257243.140625,1.010203,78.640363,0.064639,-444,1721800.0,0.147895,626.890982,0.0,65535.0,8663.987810,5985.547910,36093.0,[1],3526.0,[1],1812.0,[1],NaN,NaN,55.0,[1],11557.0,[1]


In [85]:
contacts_df

,dataset,image_name,label,organelle,X_label,org_A,org_A_label,org_B,org_B_label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,sa_volume_ratio,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length
0,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,1,lyso_mito,11_2,lyso,11,mito,2,2.363636,459.909091,380.121212,2,456,378,4,465,383,33.0,104.924561,3.179532,3.979591,0.366667,1,46.0,0.717391,8.417855
1,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,2,lyso_mito,12_2,lyso,12,mito,2,2.000000,484.450000,505.450000,2,483,503,3,487,510,20.0,73.068878,3.653444,3.367781,0.714286,1,0.0,inf,7.845296
2,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,3,lyso_mito,13_2,lyso,13,mito,2,2.863946,512.285714,460.741497,2,507,456,5,518,468,147.0,275.993652,1.877508,6.547963,0.371212,1,211.0,0.696682,11.850215
3,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,4,lyso_mito,16_2,lyso,16,mito,2,3.375000,482.312500,379.906250,3,478,378,5,486,383,32.0,100.412262,3.137883,3.938980,0.400000,1,43.0,0.744186,9.072766
4,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,5,lyso_mito,19_2,lyso,19,mito,2,3.097561,493.097561,492.878049,3,489,489,5,497,497,41.0,133.558533,3.257525,4.278207,0.320312,1,60.0,0.683333,9.596803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,90,perox_ER,88_1,perox,88,ER,1,13.000000,224.800000,422.400000,13,224,422,14,227,424,5.0,24.706015,4.941203,2.121569,0.833333,1,0.0,inf,3.464102
9775,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,91,perox_ER,56_1,perox,56,ER,1,13.000000,254.000000,413.000000,13,254,413,14,255,414,1.0,6.928203,6.928203,1.240701,1.000000,1,0.0,inf,0.000000
9776,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,92,perox_ER,97_1,perox,97,ER,1,13.000000,364.500000,253.500000,13,364,253,14,366,255,2.0,13.856404,6.928202,1.563185,0.500000,1,0.0,inf,3.162278
9777,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,93,perox_ER,98_1,perox,98,ER,1,15.000000,166.250000,349.000000,15,166,348,16,168,351,4.0,21.513260,5.378315,1.969490,0.666667,1,0.0,inf,3.162278


In [86]:
regions_df

,dataset,image_name,region,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,sa_volume_ratio,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity-0,min_intensity-1,min_intensity-2,min_intensity-3,min_intensity-4,min_intensity-5,max_intensity-0,max_intensity-1,max_intensity-2,max_intensity-3,max_intensity-4,max_intensity-5,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3,mean_intensity-4,mean_intensity-5,standard_deviation_intensity-0,standard_deviation_intensity-1,standard_deviation_intensity-2,standard_deviation_intensity-3,standard_deviation_intensity-4,standard_deviation_intensity-5
0,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,cell,1,6.197872,428.801250,335.530607,2,0,0,12,704,704,1385256.0,433195.500000,3.179532,138.306887,0.279502,5,2228899.0,0.621498,683.754058,0.0,0.0,0.0,0.0,0.0,0.0,27528.0,33141.0,65535.0,62085.0,36701.0,46708.0,403.168213,692.458477,771.805308,779.481645,1947.697575,1274.262327,1407.065577,2000.562999,3166.995343,2036.527360,2567.820770,2881.159265
1,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,cyto,1,6.219782,433.585791,345.428902,2,0,0,12,704,704,1200676.0,509387.375000,3.653444,131.868906,0.242259,6,2228899.0,0.538686,716.709526,0.0,0.0,0.0,0.0,0.0,0.0,27528.0,33141.0,65535.0,62085.0,36701.0,46708.0,425.212126,787.113542,871.315025,871.040081,2194.334357,1437.301181,1496.487741,2131.404180,3388.818724,2168.966451,2670.335772,3061.142759
2,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,nuc,2,6.040147,395.785736,270.318574,2,279,161,12,507,386,261616.0,82129.609375,1.877508,79.351516,0.509973,1,308364.0,0.848400,249.050492,0.0,0.0,0.0,0.0,0.0,0.0,12723.0,5034.0,5905.0,10281.0,8669.0,5118.0,265.768382,79.491155,128.359301,244.177481,505.640878,248.953401,539.561569,227.205225,303.515298,472.228979,642.459358,279.599798
3,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_3_Unmixing,cell,1,8.008865,327.122783,362.547316,2,37,26,15,662,704,1524566.0,446066.468750,3.137883,142.795943,0.276754,-11,2694300.0,0.565849,543.021915,0.0,0.0,0.0,0.0,0.0,0.0,12116.0,16304.0,65535.0,55322.0,39102.0,44187.0,208.885957,372.714601,702.209159,782.494696,1795.936843,890.277209,563.562522,1007.714497,2917.406918,1654.605360,2215.850078,2017.096597
4,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_3_Unmixing,cyto,1,8.151382,340.566871,360.544005,2,37,26,15,662,704,1127826.0,541597.875000,3.257525,129.146060,0.204734,-10,2694300.0,0.418597,600.987883,0.0,0.0,0.0,0.0,0.0,0.0,12116.0,16304.0,65535.0,55322.0,39102.0,44187.0,159.991511,477.686593,901.542501,923.539096,2115.190216,1108.037031,498.502926,1143.453274,3362.518542,1853.968275,2389.977348,2294.730076
5,20230922_TEST_withregions_regions,24hrs-Ctrl +oleicAcid 50uM_3_Unmixing,nuc,4,7.607422,288.263446,368.279921,2,166,241,15,399,508,490764.0,100309.101562,2.213499,97.864390,0.606823,1,528842.0,0.927997,289.646490,0.0,0.0,0.0,0.0,0.0,0.0,7186.0,5685.0,17793.0,34640.0,18218.0,13415.0,338.601768,79.439496,149.860377,449.376215,1124.335094,334.590522,677.572541,263.462604,414.816605,838.926292,1452.374604,471.895043
6,20230922_TEST_withregions_regions,a24hrs-Ctrl +oleicAcid 50uM_10_Unmixing,cell,1,10.399536,418.503591,277.745075,3,16,101,20,680,620,1308173.0,305086.531250,6.030839,135.692399,0.223296,-3,2037599.0,0.642017,530.693387,0.0,0.0,0.0,0.0,0.0,0.0,13243.0,24004.0,46100.0,50873.0,28711.0,31691.0,290.738296,439.340372,393.455995,661.129718,1437.561406,617.630372,750.983615,1362.345476,1394.062782,1652.069807,1885.350797,1195.399005
7,20230922_TEST_withregions_regions,a24hrs-Ctrl +oleicAcid 50uM_10_Unmixing,cyto,1,10.328079,413.678126,292.812644,3,16,101,20,680,620,955362.0,375950.781250,1.957694,122.195847,0.163074,-2,2037599.0,0.468867,592.228232,0.0,0.0,0.0,0.0,0.0,0.0,13243.0,24004.0,46100.0,50873.0,28711.0,31691.0,198.936236,573.565322,492.175555,818

#### Counting the number of organelles involved in contact organelles in the org_df and normalizing the count and area to the organelle area

In [87]:
column_names = org_df.columns
for col in column_names:
    if col.endswith("_labels"):
        org_df[f"n_{col.split('_')[0]}_contacting"] = org_df[col].map(len, 'ignore')

for col in column_names:
    if col.endswith("_overlap"):
        org_df[f"area_{col.split('_')[0]}_contacting_norm"] = org_df[col].div(org_df["volume"])

column_names = org_df.columns
for col in column_names:
    if col.endswith("_contacting"):
        org_df[f"n_{col.split('_')[1]}_contacting_norm"] = org_df[col].div(org_df["volume"])

In [88]:
org_df

,dataset,image_name,organelle,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,sa_volume_ratio,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity,mito_overlap,mito_labels,golgi_overlap,golgi_labels,perox_overlap,perox_labels,ER_overlap,ER_labels,LD_overlap,LD_labels,lyso_overlap,lyso_labels,n_mito_contacting,n_golgi_contacting,n_perox_contacting,n_ER_contacting,n_LD_contacting,n_lyso_contacting,area_mito_contacting_norm,area_golgi_contacting_norm,area_perox_contacting_norm,area_ER_contacting_norm,area_LD_contacting_norm,area_lyso_contacting_norm,n_mito_contacting_norm,n_golgi_contacting_norm,n_perox_contacting_norm,n_ER_contacting_norm,n_LD_contacting_norm,n_lyso_contacting_norm
0,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,3,2.445946,408.871622,361.472973,2,404,356,4,415,367,148.0,259.625702,1.754228,6.562778,0.611570,1,164.0,0.902439,13.198776,0.0,10274.0,3352.513514,2409.334613,0.0,[],0.0,[],0.0,[],114.0,[1],0.0,[],NaN,NaN,2.0,2.0,2.0,3.0,2.0,NaN,0.000000,0.000000,0.000000,0.770270,0.000000,NaN,0.013514,0.013514,0.013514,0.020270,0.013514,NaN
1,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,4,2.506410,442.634615,233.070513,2,435,228,4,452,239,156.0,393.570221,2.522886,6.678957,0.417112,1,238.0,0.655462,20.102100,0.0,12723.0,4483.294872,3374.768226,0.0,[],0.0,[],0.0,[],101.0,[1],0.0,[],NaN,NaN,2.0,2.0,2.0,3.0,2.0,NaN,0.000000,0.000000,0.000000,0.647436,0.000000,NaN,0.012821,0.012821,0.012821,0.019231,0.012821,NaN
2,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,6,2.307692,338.646154,345.938462,2,335,342,4,343,351,65.0,155.116577,2.386409,4.988518,0.451389,1,81.0,0.802469,9.003436,0.0,6547.0,1794.353846,1586.629194,0.0,[],0.0,[],0.0,[],65.0,[1],0.0,[],NaN,NaN,2.0,2.0,2.0,3.0,2.0,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,0.030769,0.030769,0.030769,0.046154,0.030769,NaN
3,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,9,2.000000,424.729167,567.520833,2,421,564,3,429,572,48.0,143.774887,2.995310,4.509007,0.750000,1,0.0,inf,9.459070,52.0,6220.0,2228.770833,1581.970583,0.0,[],0.0,[],0.0,[],10.0,[1],0.0,[],NaN,NaN,2.0,2.0,2.0,3.0,2.0,NaN,0.000000,0.000000,0.000000,0.208333,0.000000,NaN,0.041667,0.041667,0.041667,0.062500,0.041667,NaN
4,20230922_TEST_withregions_organelles,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,10,3.321348,434.123596,412.208989,2,427,406,6,443,419,445.0,498.599670,1.120449,9.472264,0.534856,1,545.0,0.816514,15.586267,0.0,14538.0,5215.573034,2812.248788,40.0,[2],0.0,[],0.0,[],32.0,[1],0.0,[],NaN,NaN,3.0,2.0,2.0,3.0,2.0,NaN,0.089888,0.000000,0.000000,0.071910,0.000000,NaN,0.006742,0.004494,0.004494,0.006742,0.004494,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,perox,97,13.000000,364.000000,254.285714,13,363,253,14,366,256,7.0,31.170115,4.452874,2.373376,0.777778,1,0.0,inf,3.380617,12025.0,23903.0,19413.000000,3858.349240,0.0,[],0.0,[],NaN,NaN,2.0,[1],0.0,[],0.0,[],2.0,2.0,NaN,3.0,2.0,2.0,0.000000,0.000000,NaN,0.285714,0.000000,0.000000,0.285714,0.285714,NaN,0.428571,0.285714,0.285714
3495,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,perox,98,15.000000,166.250000,349.000000,15,166,348,16,168,351,4.0,21.513260,5.378315,1.969490,0.666667,1,0.0,inf,3.162278,17600.0,27696.0,20437.750000,4198.654926,0.0,[],0.0,[],NaN,NaN,4.0,[1],0.0,[],0.0,[],2.0,2.0,NaN,3.0,2.0,2.0,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.500000,0.500000,NaN,0.750000,0.500000,0.500000
3496,20230922_TEST_withregions_organelles,a48hrs-Ctrl_9_Unmixing,perox,99,15.000000,329.800000,331.500000

____________________
## Summarizing per organelle metrics per cell

In [ ]:
org_df

#### Organelles: sum and count

In [122]:
stats1 = org_df[['dataset',"image_name", "organelle", "volume"]].groupby(['dataset', "image_name", "organelle"]).agg(["count", "sum"])
stats1

volume   
                                                                                      count   
dataset                              image_name                            organelle          
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER             1  \
                                                                           LD            44   
                                                                           golgi          5   
                                                                           lyso          74   
                                                                           mito           4   
...                                                                                     ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             1   
                                                                           golgi          3   
                                                                           lyso         307   
                                                                           mito           4   
                                                                           perox         96   

                                                                                                
                                                                                           sum  
dataset                              image_name                            organelle            
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.0  
                                                                           LD          11220.0  
                                                                           golgi       19293.0  
                                                                           lyso        36403.0  
                                                                           mito       123935.0  
...                                                                                        ...  
                                     a48hrs-Ctrl_9_Unmixing                LD             80.0  
                                                                           golgi       12798.0  
                                                                           lyso        42152.0  
                                                                           mito       121573.0  
                                                                           perox        3556.0  

[76 rows x 2 columns]

####  Organelles: mean, median, standard deviation

In [123]:
stats2 = org_df[['dataset',"image_name", "organelle", "volume", "surface_area", "sa_volume_ratio", "equivalent_diameter", 
                 "extent", "euler_number", "convex_area", "solidity", "axis_major_length"]].groupby(['dataset',"image_name", "organelle"]).agg(["mean", "median", "std"])
stats2

volume   
                                                                                               mean   
dataset                              image_name                            organelle                  
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.000000  \
                                                                           LD            255.000000   
                                                                           golgi        3858.600000   
                                                                           lyso          491.932432   
                                                                           mito        30983.750000   
...                                                                                             ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.000000   
                                                                           golgi        4266.000000   
                                                                           lyso          137.302932   
                                                                           mito        30393.250000   
                                                                           perox          37.041667   

                                                                                                 
                                                                                        median   
dataset                              image_name                            organelle             
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.0  \
                                                                           LD             69.0   
                                                                           golgi         889.0   
                                                                           lyso           60.0   
                                                                           mito         1566.5   
...                                                                                        ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.0   
                                                                           golgi        3035.0   
                                                                           lyso           29.0   
                                                                           mito         2632.5   
                                                                           perox          26.0   

                                                                                                     
                                                                                               std   
dataset                              image_name                            organelle                 
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER                  NaN  \
                                                                           LD          1056.365115   
                                                                           golgi       5137.722968   
                                                                           lyso        1997.782650   
                                                                           mito       59801.858078   
...                                                                                            ...   
                                     a48hrs-Ctrl_9_Unmixing                LD                  NaN   
                                                                           golgi       4254.240590   
                                                                           lyso        1081.329155   
                                                     

In [124]:
column_names = org_df.columns
org_cont_columns = ['dataset',"image_name", "organelle"] + [col for col in column_names if col.endswith(("_overlap", "_contacting", "_contacting_norm"))]

test_thing = org_df[org_cont_columns]

stats3 = test_thing.groupby(['dataset','image_name', 'organelle']).agg(['mean', 'median', 'std'])
stats3

mito_overlap   
                                                                                              mean   
dataset                              image_name                            organelle                 
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         59126.000000  \
                                                                           LD             1.363636   
                                                                           golgi       1048.200000   
                                                                           lyso          76.229730   
                                                                           mito                NaN   
...                                                                                            ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             0.000000   
                                                                           golgi        648.000000   
                                                                           lyso           8.345277   
                                                                           mito                NaN   
                                                                           perox          3.104167   

                                                                                                
                                                                                       median   
dataset                              image_name                            organelle            
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         59126.0  \
                                                                           LD             0.0   
                                                                           golgi         54.0   
                                                                           lyso           2.5   
                                                                           mito           NaN   
...                                                                                       ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             0.0   
                                                                           golgi        192.0   
                                                                           lyso           0.0   
                                                                           mito           NaN   
                                                                           perox          0.0   

                                                                                                    
                                                                                              std   
dataset                              image_name                            organelle                
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER                 NaN  \
                                                                           LD            3.369431   
                                                                           golgi      1523.022882   
                                                                           lyso        266.271288   
                                                                           mito               NaN   
...                                                                                           ...   
                                     a48hrs-Ctrl_9_Unmixing                LD                 NaN   
                                                                           golgi       938.164698   
                                                                           lyso        125.320444   
                                                                           mito       

#### Organelles: Combine summary stats into one new dataframe

In [126]:
combined_2 = pd.merge(stats1, stats2, 'outer', left_on=["dataset", "image_name", "organelle"],right_index=True)

combined_2

volume   
                                                                                      count   
dataset                              image_name                            organelle          
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER             1  \
                                                                           LD            44   
                                                                           golgi          5   
                                                                           lyso          74   
                                                                           mito           4   
...                                                                                     ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             1   
                                                                           golgi          3   
                                                                           lyso         307   
                                                                           mito           4   
                                                                           perox         96   

                                                                                                 
                                                                                           sum   
dataset                              image_name                            organelle             
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.0  \
                                                                           LD          11220.0   
                                                                           golgi       19293.0   
                                                                           lyso        36403.0   
                                                                           mito       123935.0   
...                                                                                        ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.0   
                                                                           golgi       12798.0   
                                                                           lyso        42152.0   
                                                                           mito       121573.0   
                                                                           perox        3556.0   

                                                                                                      
                                                                                               mean   
dataset                              image_name                            organelle                  
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.000000  \
                                                                           LD            255.000000   
                                                                           golgi        3858.600000   
                                                                           lyso          491.932432   
                                                                           mito        30983.750000   
...                                                                                             ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.000000   
                                                                           golgi        4266.000000   
                                                                           lyso          137.302932   
                                                                           mito        30393.250000   
                                          

In [127]:
combined_3 = pd.merge(combined_2, stats3, 'outer', left_on=["dataset", "image_name", "organelle"],right_index=True)
combined_3

volume   
                                                                                      count   
dataset                              image_name                            organelle          
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER             1  \
                                                                           LD            44   
                                                                           golgi          5   
                                                                           lyso          74   
                                                                           mito           4   
...                                                                                     ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             1   
                                                                           golgi          3   
                                                                           lyso         307   
                                                                           mito           4   
                                                                           perox         96   

                                                                                                 
                                                                                           sum   
dataset                              image_name                            organelle             
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.0  \
                                                                           LD          11220.0   
                                                                           golgi       19293.0   
                                                                           lyso        36403.0   
                                                                           mito       123935.0   
...                                                                                        ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.0   
                                                                           golgi       12798.0   
                                                                           lyso        42152.0   
                                                                           mito       121573.0   
                                                                           perox        3556.0   

                                                                                                      
                                                                                               mean   
dataset                              image_name                            organelle                  
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.000000  \
                                                                           LD            255.000000   
                                                                           golgi        3858.600000   
                                                                           lyso          491.932432   
                                                                           mito        30983.750000   
...                                                                                             ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.000000   
                                                                           golgi        4266.000000   
                                                                           lyso          137.302932   
                                                                           mito        30393.250000   
                                          

#### Organelles: testing out easier list method of making combos

In [129]:
group_by_org = ['dataset','image_name', 'organelle']

columns1 = ['volume', 'surface_area', 'sa_volume_ratio', 'equivalent_diameter', 'extent',
            'euler_number',	'convex_area', 'solidity', 'axis_major_length']
columns2 = [col for col in column_names if col.endswith(("_overlap", "_contacting", "_contacting_norm"))]

ag_func_standard = ['mean', 'median', 'std']

# organelles
summary_tab_list = []
summary_tab_list.append(org_df[group_by_org + ['volume']].groupby(group_by_org).agg(['count', 'sum']))
summary_tab_list.append(org_df[group_by_org + columns1 + columns2].groupby(group_by_org).agg(ag_func_standard))

test_merge = pd.merge(summary_tab_list[0], summary_tab_list[1], 'outer', left_on=["dataset", "image_name", "organelle"],right_index=True)
test_merge

volume   
                                                                                      count   
dataset                              image_name                            organelle          
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER             1  \
                                                                           LD            44   
                                                                           golgi          5   
                                                                           lyso          74   
                                                                           mito           4   
...                                                                                     ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             1   
                                                                           golgi          3   
                                                                           lyso         307   
                                                                           mito           4   
                                                                           perox         96   

                                                                                                 
                                                                                           sum   
dataset                              image_name                            organelle             
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.0  \
                                                                           LD          11220.0   
                                                                           golgi       19293.0   
                                                                           lyso        36403.0   
                                                                           mito       123935.0   
...                                                                                        ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.0   
                                                                           golgi       12798.0   
                                                                           lyso        42152.0   
                                                                           mito       121573.0   
                                                                           perox        3556.0   

                                                                                                      
                                                                                               mean   
dataset                              image_name                            organelle                  
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.000000  \
                                                                           LD            255.000000   
                                                                           golgi        3858.600000   
                                                                           lyso          491.932432   
                                                                           mito        30983.750000   
...                                                                                             ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.000000   
                                                                           golgi        4266.000000   
                                                                           lyso          137.302932   
                                                                           mito        30393.250000   
                                          

In [130]:
combined_3

volume   
                                                                                      count   
dataset                              image_name                            organelle          
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER             1  \
                                                                           LD            44   
                                                                           golgi          5   
                                                                           lyso          74   
                                                                           mito           4   
...                                                                                     ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             1   
                                                                           golgi          3   
                                                                           lyso         307   
                                                                           mito           4   
                                                                           perox         96   

                                                                                                 
                                                                                           sum   
dataset                              image_name                            organelle             
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.0  \
                                                                           LD          11220.0   
                                                                           golgi       19293.0   
                                                                           lyso        36403.0   
                                                                           mito       123935.0   
...                                                                                        ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.0   
                                                                           golgi       12798.0   
                                                                           lyso        42152.0   
                                                                           mito       121573.0   
                                                                           perox        3556.0   

                                                                                                      
                                                                                               mean   
dataset                              image_name                            organelle                  
20230922_TEST_withregions_organelles 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER         354310.000000  \
                                                                           LD            255.000000   
                                                                           golgi        3858.600000   
                                                                           lyso          491.932432   
                                                                           mito        30983.750000   
...                                                                                             ...   
                                     a48hrs-Ctrl_9_Unmixing                LD             80.000000   
                                                                           golgi        4266.000000   
                                                                           lyso          137.302932   
                                                                           mito        30393.250000   
                                          

In [131]:
test_merge.equals(combined_3)

True

#### Organelles + distributions: adding the distribution columns onto the combined organelle summary sheet

In [141]:
dist_df

,dataset,image_name,organelle,XY_n_bins,XY_bins,XY_cm_vox_cnt,XY_org_vox_cnt,XY_n_pix,XY_cm_cv,XY_org_cv,zernike_n,zernike_m,zernike_cm_mag,zernike_cm_phs,zernike_obj_mag,zernike_obj_phs,zernike_nuc_mag,zernike_nuc_phs,z_n,z_slice,z_cm_vox_cnt,z_org_vox_cnt,z_nuc_vox_cnt
0,20230922_TEST_withregions_distributions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,lyso,5,"['1', '2', '3', '4', '5']","[490488, 155575, 206973, 268364, 263856]","[2413, 8882, 15622, 8706, 780]","[49051.0, 15903.0, 22985.0, 40326.0, 74732.0]","[8.321501485258628e-05, 0.040054507538727704, ...","[1.0365467820989047, 1.603271723271731, 1.6689...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.2770483979997788, 0.5461988487672376, ...","[1.5707963267948966, -0.12479380508422404, -1....","[1.0, 0.4981931017165809, 0.43744093760703445,...","[1.5707963267948966, 0.11947973247455637, -1.5...","[1.0, 0.2686711517263515, 0.8053485198587096, ...","[1.5707963267948966, -0.7108541432185989, -1.5...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 162424, 163166, 153092, 146649, 133633,...","[0, 0, 1043, 2044, 3467, 4510, 4813, 5014, 503...","[0, 0, 11271, 28253, 34525, 37621, 37884, 3699..."
1,20230922_TEST_withregions_distributions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,mito,5,"['1', '2', '3', '4', '5']","[490488, 155575, 206973, 268364, 263856]","[17135, 30528, 26708, 35351, 14213]","[49051.0, 15903.0, 22985.0, 40326.0, 74732.0]","[8.321501485258628e-05, 0.040054507538727704, ...","[1.1304118818106073, 0.697806108479738, 0.7583...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.2770483979997788, 0.5461988487672376, ...","[1.5707963267948966, -0.12479380508422404, -1....","[1.0, 0.3210208546974688, 0.5028630951789582, ...","[1.5707963267948966, 0.04050341070446073, -1.5...","[1.0, 0.2686711517263515, 0.8053485198587096, ...","[1.5707963267948966, -0.7108541432185989, -1.5...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 162424, 163166, 153092, 146649, 133633,...","[0, 0, 9092, 13368, 15928, 15433, 14908, 14368...","[0, 0, 11271, 28253, 34525, 37621, 37884, 3699..."
2,20230922_TEST_withregions_distributions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,golgi,5,"['1', '2', '3', '4', '5']","[490488, 155575, 206973, 268364, 263856]","[5269, 6726, 2841, 4451, 6]","[49051.0, 15903.0, 22985.0, 40326.0, 74732.0]","[8.321501485258628e-05, 0.040054507538727704, ...","[2.0877453890531963, 2.5327744142694724, 2.485...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.2770483979997788, 0.5461988487672376, ...","[1.5707963267948966, -0.12479380508422404, -1....","[1.0, 0.4292588948394096, 0.6119414527276666, ...","[1.5707963267948966, 0.3592321743340551, -1.57...","[1.0, 0.2686711517263515, 0.8053485198587096, ...","[1.5707963267948966, -0.7108541432185989, -1.5...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 162424, 163166, 153092, 146649, 133633,...","[0, 0, 479, 1182, 1878, 3449, 4129, 3707, 2503...","[0, 0, 11271, 28253, 34525, 37621, 37884, 3699..."
3,20230922_TEST_withregions_distributions,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,perox,5,"['1', '2', '3', '4', '5']","[490488, 155575, 206973, 268364, 263856]","[873, 1328, 769, 1528, 818]","[49051.0, 15903.0, 22985.0, 40326.0, 74732.0]","[8.321501485258628e-05, 0.040054507538727704, ...","[0.6928982377133949, 0.8797540125944158, 0.570...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.2770483979997788, 0.5461988487672376, ...","[1.5707963267948966, -0.12479380508422404, -1....","[1.0, 0.3159681981136953, 0.5008125126319019, ...","[1.5707963267948966, -0.16432271969840623, -1....","[1.0, 0.2686711517263515, 0.8053485198587096, ...","[1.5707963267948966, -0.7108541432185989, -1.5...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0

In [147]:
group_by_org = ['dataset', 'image_name', 'organelle']

dist1 = dist_df[group_by_org + ['XY_cm_vox_cnt','XY_org_vox_cnt',
                            'zernike_cm_phs','zernike_obj_mag','zernike_obj_phs','z_cm_vox_cnt',
                            'z_org_vox_cnt','z_nuc_vox_cnt']].set_index(["dataset", "image_name", "organelle"])
dist1

XY_cm_vox_cnt   
dataset                                 image_name                            organelle                                             
20230922_TEST_withregions_distributions 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing lyso       [490488, 155575, 206973, 268364, 263856]  \
                                                                              mito       [490488, 155575, 206973, 268364, 263856]   
                                                                              golgi      [490488, 155575, 206973, 268364, 263856]   
                                                                              perox      [490488, 155575, 206973, 268364, 263856]   
                                                                              ER         [490488, 155575, 206973, 268364, 263856]   
...                                                                                                                           ...   
                                        a48hrs-Ctrl_9_Unmixing                golgiXER     [480492, 77726, 92499, 182794, 270489]   
                                                                              golgiXLD     [480492, 77726, 92499, 182794, 270489]   
                                                                              peroxXER     [480492, 77726, 92499, 182794, 270489]   
                                                                              peroxXLD     [480492, 77726, 92499, 182794, 270489]   
                                                                              ERXLD        [480492, 77726, 92499, 182794, 270489]   

                                                                                                              XY_org_vox_cnt   
dataset                                 image_name                            organelle                                        
20230922_TEST_withregions_distributions 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing lyso            [2413, 8882, 15622, 8706, 780]  \
                                                                              mito       [17135, 30528, 26708, 35351, 14213]   
                                                                              golgi              [5269, 6726, 2841, 4451, 6]   
                                                                              perox              [873, 1328, 769, 1528, 818]   
                                                                              ER         [82642, 61738, 66752, 82170, 61008]   
...                                                                                                                      ...   
                                        a48hrs-Ctrl_9_Unmixing                golgiXER            [685, 825, 1283, 445, 288]   
                                                                              golgiXLD                       [0, 0, 0, 0, 0]   
                                                                              peroxXER             [450, 207, 289, 470, 396]   
                                                                              peroxXLD                       [0, 0, 0, 0, 0]   
                                                                              ERXLD                        [20, 35, 0, 0, 0]   

                                                                                                                            zernike_cm_phs   
dataset                                 image_name                            organelle                                                      
20230922_TEST_withregions_distributions 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing lyso       [1.5707963267948966, -0.12479380508422404, -1....  \
                                                                              mito       [1.5707963267948966, -0.12479380508422404, -1....   
                                                                              golgi      [1.5707963267948966, -0.12479380508422404, -1....   
           

In [148]:
org_dist_merge = pd.merge(test_merge, dist1, 'outer', left_on=["dataset", "image_name", "organelle"], right_index=True)

MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)

#### Contacts & cell regions: ^^^ similar will be done to sum, count, mean, median, and standard deviation 

In [132]:
contacts_df

,dataset,image_name,label,organelle,X_label,org_A,org_A_label,org_B,org_B_label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,sa_volume_ratio,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length
0,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,1,lyso_mito,11_2,lyso,11,mito,2,2.363636,459.909091,380.121212,2,456,378,4,465,383,33.0,104.924561,3.179532,3.979591,0.366667,1,46.0,0.717391,8.417855
1,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,2,lyso_mito,12_2,lyso,12,mito,2,2.000000,484.450000,505.450000,2,483,503,3,487,510,20.0,73.068878,3.653444,3.367781,0.714286,1,0.0,inf,7.845296
2,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,3,lyso_mito,13_2,lyso,13,mito,2,2.863946,512.285714,460.741497,2,507,456,5,518,468,147.0,275.993652,1.877508,6.547963,0.371212,1,211.0,0.696682,11.850215
3,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,4,lyso_mito,16_2,lyso,16,mito,2,3.375000,482.312500,379.906250,3,478,378,5,486,383,32.0,100.412262,3.137883,3.938980,0.400000,1,43.0,0.744186,9.072766
4,20230922_TEST_withregions_contacts,24hrs-Ctrl +oleicAcid 50uM_2_Unmixing,5,lyso_mito,19_2,lyso,19,mito,2,3.097561,493.097561,492.878049,3,489,489,5,497,497,41.0,133.558533,3.257525,4.278207,0.320312,1,60.0,0.683333,9.596803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,90,perox_ER,88_1,perox,88,ER,1,13.000000,224.800000,422.400000,13,224,422,14,227,424,5.0,24.706015,4.941203,2.121569,0.833333,1,0.0,inf,3.464102
9775,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,91,perox_ER,56_1,perox,56,ER,1,13.000000,254.000000,413.000000,13,254,413,14,255,414,1.0,6.928203,6.928203,1.240701,1.000000,1,0.0,inf,0.000000
9776,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,92,perox_ER,97_1,perox,97,ER,1,13.000000,364.500000,253.500000,13,364,253,14,366,255,2.0,13.856404,6.928202,1.563185,0.500000,1,0.0,inf,3.162278
9777,20230922_TEST_withregions_contacts,a48hrs-Ctrl_9_Unmixing,93,perox_ER,98_1,perox,98,ER,1,15.000000,166.250000,349.000000,15,166,348,16,168,351,4.0,21.513260,5.378315,1.969490,0.666667,1,0.0,inf,3.162278


In [135]:
group_by = ["dataset", 'image_name', 'organelle']

columns1 = ['volume', 'surface_area', 'sa_volume_ratio', 'equivalent_diameter', 'extent',
            'euler_number',	'convex_area', 'solidity', 'axis_major_length']
columns2 = [col for col in column_names if col.endswith(("_overlap", "_contacting", "_contacting_norm"))]

ag_func_standard = ['mean', 'median', 'std']

In [138]:
contact_1 = contacts_df[group_by +['volume']].groupby(group_by).agg(["count", "sum"])
contact_2 = contacts_df[group_by + columns1].groupby(group_by).agg(ag_func_standard)
contact_merged = pd.merge(contact_1, contact_2, 'outer', left_on=["dataset", "image_name", "organelle"],right_index=True)
contact_merged

volume   
                                                                                      count   
dataset                            image_name                            organelle            
20230922_TEST_withregions_contacts 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER_LD           49  \
                                                                         golgi_ER        33   
                                                                         golgi_LD         4   
                                                                         golgi_perox      3   
                                                                         lyso_ER        130   
...                                                                                     ...   
                                   a48hrs-Ctrl_9_Unmixing                lyso_perox      11   
                                                                         mito_ER        170   
                                                                         mito_golgi      30   
                                                                         mito_perox      32   
                                                                         perox_ER        93   

                                                                                                
                                                                                          sum   
dataset                            image_name                            organelle              
20230922_TEST_withregions_contacts 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER_LD         3794.0  \
                                                                         golgi_ER      7235.0   
                                                                         golgi_LD        73.0   
                                                                         golgi_perox     24.0   
                                                                         lyso_ER      12675.0   
...                                                                                       ...   
                                   a48hrs-Ctrl_9_Unmixing                lyso_perox      78.0   
                                                                         mito_ER      36093.0   
                                                                         mito_golgi    1944.0   
                                                                         mito_perox     298.0   
                                                                         perox_ER      1812.0   

                                                                                                   
                                                                                            mean   
dataset                            image_name                            organelle                 
20230922_TEST_withregions_contacts 24hrs-Ctrl +oleicAcid 50uM_2_Unmixing ER_LD         77.428571  \
                                                                         golgi_ER     219.242424   
                                                                         golgi_LD      18.250000   
                                                                         golgi_perox    8.000000   
                                                                         lyso_ER       97.500000   
...                                                                                          ...   
                                   a48hrs-Ctrl_9_Unmixing                lyso_perox     7.090909   
                                                                         mito_ER      212.311765   
                                                                         mito_golgi    64.800000   
                                                                         mito_perox     9.312500   
                                                                         perox_ER      19.48387

#### Distributions: will add the table on based on dataset, image_name, and organelle

####  Organelles: fixing column and index labels

In [ ]:
stats3.columns = ["_".join(col_name).rstrip('_') for col_name in stats3.columns.to_flat_index()]
stats3.reset_index()

#### `WIP still`
#### Quality control tabels - intensities

In [ ]:
result = pd.concat([math_summary, vol_summary, overlap_summary, label_summary], axis=1)

In [ ]:
# adding these to a quality control sheet
"min_intensity","max_intensity","mean_intensity","standard_deviation_intensity"

## Create combined function to do the above

In [ ]:
def _batch_summary_stats(csv_path_list: List[str],
                         out_path: str,
                         out_preffix: str):
    """" 
    csv_path_list: List[str],
        A list of path strings where .csv files to analyze are located.
    out_path: str,
        A path string where the summary data file will be output to
    out_preffix: str
        The prefix used to name the output file.    
    """
    ds_count = 0
    fl_count = 0
    ###################
    # Read in the csv files and combine them into one of each type
    ###################
    org_tabs = []
    contact_tabs = []
    dist_tabs = []
    region_tabs = []

    for loc in csv_path_list:
        ds_count = ds_count + 1
        files_store = sorted(loc.glob("*.csv"))
        for file in files_store:
            fl_count = fl_count + 1
            stem = file.stem.split("-")[0]

            org = "organelles"
            contacts = "contacts"
            dist = "distributions"
            regions = "regions"

            if org in stem:
                test_orgs = pd.read_csv(file, index_col=0)
                test_orgs.insert(0, "dataset", stem)
                org_tabs.append(test_orgs)
            if contacts in stem:
                test_contact = pd.read_csv(file, index_col=0)
                test_contact.insert(0, "dataset", stem)
                contact_tabs.append(test_contact)
            if dist in stem:
                test_dist = pd.read_csv(file, index_col=0)
                test_dist.insert(0, "dataset", stem)
                dist_tabs.append(test_dist)
            if regions in stem:
                test_regions = pd.read_csv(file, index_col=0)
                region_tabs.append(test_regions)
            

    org_df = pd.concat(org_tabs,axis=0, join='outer')
    contacts_df = pd.concat(contact_tabs,axis=0, join='outer')
    dist_df = pd.concat(dist_tabs,axis=0, join='outer')
    # regions_df = pd.concat(region_tabs,axis=0, join='outer')

    ###################
    # table math
    ###################
    # regions_df.rename(columns={"region": "organelle"}, inplace=True)
    contacts_df.insert(3, "organelle", (contacts_df['org_A'] + '_' + contacts_df['org_B']))
    
    org_df.insert(14, "sa_volume_ratio", org_df["surface_area"].div(org_df["volume"]))
    contacts_df.insert(17, "sa_volume_ratio", contacts_df["surface_area"].div(contacts_df["volume"]))
    # regions_df.insert(14, "sa_volume_ratio", contacts_df["surface_area"].div(contacts_df["volume"]))

    column_names = org_df.columns
    for col in column_names:
        if col.endswith("_labels"):
            org_df[f"n_{col.split('_')[0]}_contacting"] = org_df[col].map(len, 'ignore')
        if col.endswith("_overlap"):
            org_df[f"area_{col.split('_')[0]}_contacting_norm"] = org_df[col].div(org_df["volume"])
    
    column_names = org_df.columns
    for col in column_names:
        if col.endswith("_contacting"):
            org_df[f"n_{col.split('_')[1]}_contacting_norm"] = org_df[col].div(org_df["volume"])

    ###################
    # summarize data per cell
    ###################
    group_by = ['image_name', 'organelle']

    columns1 = ['sa_volume_ratio', 'volume', 'surface_area', 'equivalent_diameter', 'extent',
                'euler_number',	'convex_area', 'solidity', 'axis_major_length']
    columns2 = [col for col in column_names if col.endswith(("_overlap", "_contacting", "_contacting_norm"))]
    columns3 = ['XY_cm_vox_cnt','XY_org_vox_cnt',
                'zernike_cm_mag','zernike_cm_phs','zernike_obj_mag','zernike_obj_phs',
                'z_cm_vox_cnt','z_org_vox_cnt','z_nuc_vox_cnt'] # ,'z_cyto_vox_cnt']

    ag_func_standard = ['mean', 'median', 'std']

    tabs = []
    # regions
    # tabs.append(regions_df[group_by_org + columns1].groupby(group_by).agg(ag_func_standard))

    # organelles
    tabs.append(org_df[group_by + ['volume']].groupby(group_by).agg(['count', 'sum']))
    tabs.append(org_df[group_by + columns1 + columns2].groupby(group_by).agg(ag_func_standard))

    # contacts
    tabs.append(contacts_df[group_by + ['volume']].groupby(group_by).agg(['count', 'sum']))
    tabs.append(contacts_df[group_by + columns1].groupby(group_by).agg(ag_func_standard))

    # distributions
    tabs.append(dist_df[group_by + columns3].reset_index(drop=True))

    # per_cell_df = pd.concat(tabs)

    # ###################
    # # export summary sheets
    # ###################
    # csv_path = out_path / f"{out_preffix}stats_per_cell.csv"
    # per_cell_df.to_csv(csv_path)


    # print(f"Processing of {fl_count} files from {ds_count} datasets is complete.")
    # return f"{fl_count} files from {ds_count} datasets were processed"

    return tabs

In [ ]:
test_multi_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc/quant"

csv_path_list = [test_multi_root_path/"test_loc_1", test_multi_root_path/"test_loc_2" ]

csv_path_list

In [ ]:
tabs = _batch_summary_stats(csv_path_list, test_multi_root_path, "20230922_test1_")

In [ ]:
pd.concat(tabs)

In [ ]:
pd.concat(tabs, join='outer',levels=['image_name', 'organelle'])

In [ ]:
len(tabs)

In [ ]:
tabs[0]

In [ ]:
tabs[1]

In [ ]:
tabs[2]

In [ ]:
tabs[3]

In [ ]:
tabs[4]

## Andy's original codes

In [ ]:
def summarize_by_id(stats_in:pd.DataFrame,agg_fn: List) -> pd.DataFrame:
    """ 
    """
    summary = stats_in.groupby(['ID']).agg(agg_fn)
    summary.columns = ["_".join(col_name).rstrip('_') for col_name in summary.columns.to_flat_index()]
    return summary

In [ ]:
def create_stats_summary(summary_df:pd.DataFrame) -> pd.DataFrame:
    """
    """
    column_names = summary_df.columns

    def frac(x):
        return (x>0).sum()/x.count() 

    math_cols = ['ID', 'mean_intensity',
        'standard_deviation_intensity',
        'min_intensity','max_intensity', 'equivalent_diameter',
        'euler_number', 'extent']
    vol_cols = ['ID','volume']
    overlap_cols = ['ID'] + [col for col in column_names if col.endswith('_overlap')]
    labels_cols = ['ID'] + [col for col in column_names if col.endswith('_labels')]
   
    agg_func_math = ['sum', 'mean', 'median', 'min', 'max', 'std','count']
    agg_func_overlap = ['sum', 'mean', 'median','count',frac]
    agg_func_labels = ['sum']
    agg_func_vol = ['sum', 'mean', 'median', 'min', 'max', 'std', 'var']

    math_summary = summarize_by_id( summary_df[math_cols] , agg_func_math)
    
    # label_stats = fix_list_col(summary_df[labels_cols])
    label_summary = summarize_by_id( summary_df[labels_cols] , agg_func_labels)
    overlap_summary = summarize_by_id( summary_df[overlap_cols] ,agg_func_overlap)
    vol_summary = summarize_by_id( summary_df[vol_cols] , agg_func_vol)
    result = pd.concat([math_summary, vol_summary, overlap_summary, label_summary], axis=1)

    result.insert(loc=0,column="ID",value=result.index)

    return result

In [ ]:
# for a list of "prefixes"  collect stats + cross stats masked by cytosol (including nuclei masked by cellmask)

def _summarize_organelle_stats(int_path: Union[Path,str], 
                              organelle_names: List[str]):
    """  
    """
    # write out files... 

    if isinstance(int_path, str): int_path = Path(int_path)

    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")

    all_stats_df = pd.DataFrame()
    all_cross_stats_df = pd.DataFrame()
    all_proj_stats_df = pd.DataFrame()
    
    for target in organelle_names:
        stat_file_list = sorted( int_path.glob(f"*{target}-stats.csv") )

        stats_df = pd.DataFrame()
        cross_stats_df = pd.DataFrame()
        proj_stats_df = pd.DataFrame()

        for stats_f in stat_file_list:
            stem = stats_f.stem.split("-")[0]
            # stats load the csv
            stats = load_stats_csv(out_path,stem, target)
            # projection stats
            proj = load_proj_stats_csv(out_path,stem, target)
            # cross stats
            cross = load_cross_stats_csv(out_path,stem, target)

            stats_df = pd.concat([stats_df,stats],axis=0, join='outer')
            proj_stats_df = pd.concat([proj_stats_df,proj],axis=0, join='outer')
            cross_stats_df = pd.concat([cross_stats_df,cross],axis=0, join='outer')
        

        ## maybe merge into all the possible files?
        # summary_df = pd.DataFrame(index=[f.stem.split("-")[0] for f in stat_file_list])
        # cross_stats_df = pd.DataFrame(index=[f.stem.split("-")[0] for f in stat_file_list])
        # proj_stats_df = pd.DataFrame(index=[f.stem.split("-")[0] for f in stat_file_list])

        summary_df = create_stats_summary(stats_df)
        summary_df.insert(loc=1,column="organelle",value=target)
        cross_summary_df = summarize_cross_stats(cross_stats_df)
        ## cross_summary_df = pivot_cross_stats(cross_stats_df)  #makes a wide version... but has a bug
        cross_summary_df.insert(loc=1,column="organelle",value=target)

        all_stats_df = pd.concat([all_stats_df,summary_df],axis=0)
        all_proj_stats_df = pd.concat([all_proj_stats_df,proj_stats_df],axis=0)
        all_cross_stats_df = pd.concat([all_cross_stats_df,cross_summary_df],axis=0)
    

    return all_stats_df, all_proj_stats_df, all_cross_stats_df
        



In [ ]:
from infer_subc.utils.stats_helpers import summarize_organelle_stats, dump_organelle_summary_tables

# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"
# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "raw"
# save output ".tiff" files here could be different than out
int_data_path = data_root_path / "out"
# save stats here
out_data_path = data_root_path / "out"


all_stats_df, all_proj_stats_df, all_cross_stats_df = summarize_organelle_stats( int_path, organelle_names )


In [ ]:
def _dump_organelle_summary_tables(
                    int_path: Union[Path,str], 
                    out_path: Union[Path, str], 
                    organelle_names: List[str]= ["nuclei","golgi","peroxi"] ) -> int:
    """
    get summary and all cross stats between organelles `a` and `b`
    calls `get_summary_stats_3D`
    """

    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")


    all_stats_df, all_proj_stats_df, all_cross_stats_df = summarize_organelle_stats( int_path, organelle_names )

    csv_path = out_path / f"summary-stats.csv"
    all_stats_df.to_csv(csv_path)

    csv_path = out_path / f"summary-proj-stats.csv"
    all_proj_stats_df.to_csv(csv_path)

    csv_path = out_path / f"summary-cross-stats.csv"
    all_cross_stats_df.to_csv(csv_path)

    return 1



In [ ]:
n_files = dump_organelle_summary_tables(out_data_path, 
                     out_data_path, 
                     organelle_names)

n_files

Make some wrappers to deal with reading our summary stats into pandas properly.


In [ ]:


def load_summary_stats_csv(in_path: Path) -> pd.DataFrame:
    """ helper to load the summary stats csv: summary-stats.csv
    returns pandas DataFrame """
    csv_path = in_path / f"summary-stats.csv"
    summary_df = pd.read_csv(csv_path, index_col=0)
    # need to convert columns *_labels
    list_cols = [col for col in summary_df.columns if "labels" in col] #if col.contains("label")
    summary_df = fix_int_list_cols(summary_df,list_cols)
    return summary_df


def load_summary_proj_stats_csv(in_path: Path) -> pd.DataFrame:
    """ helper to load summary projection stats csv: summary-proj-stats.csv
    returns pandas DataFrame """
    obj_cols =  ['ID', 'organelle','mask','radial_n_bins','n_z']  # leave alone
    str_cols = [ 'radial_bins']
    int_cols = ['radial_cm_vox_cnt', 'radial_org_vox_cnt', 'radial_org_intensity', 'radial_n_pix','zernike_n', 'zernike_m', 'z','z_cm_vox_cnt','z_org_vox_cnt', 'z_org_intensity', 'z_nuc_vox_cnt']
    float_cols = ['radial_cm_cv', 'radial_org_cv', 'radial_img_cv','zernike_cm_mag', 'zernike_cm_phs','zernike_obj_mag', 'zernike_obj_phs', 'zernike_nuc_mag','zernike_nuc_phs', 'zernike_img_mag']

    csv_path = in_path / f"summary-proj-stats.csv"
    proj = pd.read_csv(csv_path, index_col=0)
    proj = fix_str_list_cols(proj, str_cols)
    proj = fix_int_list_cols(proj, int_cols)
    proj = fix_float_list_cols(proj, float_cols)
    return proj
        

def load_summary_cross_stats_csv(in_path: Path) -> pd.DataFrame:
    """ helper to load summary cross- stats csv: summary-cross-stats.csv
    returns pandas DataFrame """

    csv_path = in_path / f"summary-cross-stats.csv"
    summary_df = pd.read_csv(csv_path, index_col=0)

    list_cols = [col for col in summary_df.columns if "label" in col] #if col.contains("label")
    str_list_cols = [col for col in list_cols if "__" in col]
    int_list_cols = [col for col in list_cols if "__" not in col]

    summary_df = fix_str_list_cols(summary_df,str_list_cols)
    summary_df = fix_int_list_cols(summary_df,int_list_cols)

    return summary_df
    


In [ ]:

#summary_shell.head()
test = load_summary_stats_csv(out_data_path)
test_proj = load_summary_proj_stats_csv(out_data_path)
test_cross = load_summary_cross_stats_csv(out_data_path)